### Hard Sudoku Solver

Write a function that will solve a 9x9 Sudoku puzzle. The function will take one argument consisting of the 2D puzzle array, with the value 0 representing an unknown square.

The Sudokus tested against your function will be "insane" and can have multiple solutions. 
The solution only need to give one valid solution in the case of the multiple solution sodoku.

In [1]:
SIZE = 9
ROWS = 'abcdefghi'
COLS = '123456789'

flat_map = lambda f, xs: sum(map(f, xs), [])

def cartesian(a1, a2):
     return [i+j for i in a1 for j in a2]

def split(string, n):
    return [ string[i:i+n] for i in range(0, len(string), n)]

def create_units(unitlist, boxes):
    """Create box to units it belongs."""
    box_units = lambda box: list(filter(lambda unit: box in unit, unitlist))
    return { box: box_units(box) for box in boxes}
    
def create_peers(units, boxes):
    """Create box to peers list map for every box"""
    unit_peers = lambda key_box, unit: list(filter(lambda b: b != key_box, unit))
    box_peers = lambda key_box: flat_map(lambda u: unit_peers(key_box, u) ,units[key_box])
    return { box: box_peers(box) for box in boxes }
    


In [2]:
boxes = cartesian(ROWS, COLS)
row_unit_split = split(ROWS, 3) # split rows by 3 to for unit 3x3
col_unit_split = split(COLS, 3) # split columns by 3 to for unit 3x3
row_units = [cartesian(r, COLS) for r in ROWS] # row of boxes as unit 
column_units = [cartesian(ROWS, c) for c in COLS] # column of boxes as unit
square_units = [cartesian(rs, cs) for rs in row_unit_split for cs in col_unit_split] # 3x3 square of boxes as unit
unitlist = row_units + column_units + square_units # list of units square, rows, columns

units = create_units(unitlist, boxes) # create map of units for every box 
peers = create_peers(units, boxes)    # create map of peers for every box

In [3]:
peers 

{'a1': ['a2',
  'a3',
  'a4',
  'a5',
  'a6',
  'a7',
  'a8',
  'a9',
  'b1',
  'c1',
  'd1',
  'e1',
  'f1',
  'g1',
  'h1',
  'i1',
  'a2',
  'a3',
  'b1',
  'b2',
  'b3',
  'c1',
  'c2',
  'c3'],
 'a2': ['a1',
  'a3',
  'a4',
  'a5',
  'a6',
  'a7',
  'a8',
  'a9',
  'b2',
  'c2',
  'd2',
  'e2',
  'f2',
  'g2',
  'h2',
  'i2',
  'a1',
  'a3',
  'b1',
  'b2',
  'b3',
  'c1',
  'c2',
  'c3'],
 'a3': ['a1',
  'a2',
  'a4',
  'a5',
  'a6',
  'a7',
  'a8',
  'a9',
  'b3',
  'c3',
  'd3',
  'e3',
  'f3',
  'g3',
  'h3',
  'i3',
  'a1',
  'a2',
  'b1',
  'b2',
  'b3',
  'c1',
  'c2',
  'c3'],
 'a4': ['a1',
  'a2',
  'a3',
  'a5',
  'a6',
  'a7',
  'a8',
  'a9',
  'b4',
  'c4',
  'd4',
  'e4',
  'f4',
  'g4',
  'h4',
  'i4',
  'a5',
  'a6',
  'b4',
  'b5',
  'b6',
  'c4',
  'c5',
  'c6'],
 'a5': ['a1',
  'a2',
  'a3',
  'a4',
  'a6',
  'a7',
  'a8',
  'a9',
  'b5',
  'c5',
  'd5',
  'e5',
  'f5',
  'g5',
  'h5',
  'i5',
  'a4',
  'a6',
  'b4',
  'b5',
  'b6',
  'c4',
  'c5',
  'c6'],
 'a6'

In [37]:
box_len = lambda b, values: len(values[b])
is_solved = lambda b, values: box_len(b, values) == 1
is_not_solved = lambda b, values: box_len(b, values) > 1

solved_count = lambda sudoku: len([b for b in boxes if is_solved(b, sudoku)])
is_puzzle_solved = lambda sudoku: not next(filter(lambda b: len(sudoku[b]) > 1, boxes), None)
find_min_unsolved = lambda sudoku: min(map(lambda b: (len(sudoku[b]), b), filter(lambda b: len(sudoku[b]) > 1, boxes)))

In [5]:
def eliminate(sudoku):
    """Eliminate solved boxes"""
    solved = filter(lambda b: is_solved(b, sudoku), boxes)
    solved_peers = map(lambda b: [sudoku[b], peers[b]], solved) 
    for d, b_peers in solved_peers:
        for p in b_peers:
            sudoku[p] = sudoku[p].replace(d, '')
    return sudoku

In [6]:
def only_choice(sudoku):
    """
    The only choice strategy says that if only one box in a unit allows a certain
    digit, then that box must be assigned that digit.
    """
    digit_in_boxes = lambda digit, unit: [box for box in unit if digit in sudoku[box]]
    for unit in unitlist:
        for digit in COLS:
            dlist = digit_in_boxes(digit, unit)
            if len(dlist) == 1:
                sudoku[dlist[0]] = digit
    return sudoku

In [28]:
def reduce_puzzle(sudoku):
    """Reduce a puzzle by repeatedly applying all constraint strategies"""
    stalled = False
    i = 0
    while not stalled:
        solved_before = solved_count(sudoku)
        sudoku = eliminate(sudoku)
        sudoku = only_choice(sudoku)
        solved_after = solved_count(sudoku)
        # If no new values were added, stop the loop.
        stalled = solved_before == solved_after
        # check if sudoku still correct
        if len([b for b in boxes if len(sudoku[b]) == 0]):
            return False
        i = i + 1
    print(f"count of reductions {i}")
    return sudoku

In [38]:
def search(sudoku):
    """Apply depth first search to solve Sudoku puzzles in order to solve puzzles that cannot be solved by repeated reduction alone. """
    # Reduce
    sudoku = reduce_puzzle(sudoku)
    if sudoku is False:
        return False
    is_any_unresolved = is_puzzle_solved(sudoku) 
    if is_puzzle_solved(sudoku): 
        return sudoku
    # Take a unsolved box with minimum posibilities
    n,b = find_min_unsolved(sudoku)
    # Now use recurrence to solve each one of the resulting sudokus, and 
    for v in sudoku[b]:
        new_sudoku = sudoku.copy()
        new_sudoku[b] = v
        attempt = search(new_sudoku)
        if attempt:
            return attempt


In [39]:

def pos(r,c):
    return f"{ROWS[r]}{COLS[c]}"

def val(v):
    return COLS if v == 0 else str(v)

def read(puzzle):
    return { pos(r,c) : val(puzzle[r][c]) for r in range(0, SIZE) for c in range(0, SIZE) }

def key_to_pos(k):
    return ROWS.find(k[0]), int(k[1])-1

def to_matrix(sudoku):
    m = [ [0]*SIZE for i in range(0, SIZE)]
    for k,v in sudoku.items():
        r,c = key_to_pos(k)
        m[r][c] = int(v)
    return m
        

def solve(puzzle):
    sudoku = read(puzzle)
    sudoku = search(sudoku)
    #sudoku = reduce_puzzle(sudoku)
    #print(sudoku)
    print(to_matrix(sudoku))

In [40]:
puzzle = [[5,3,0,0,7,0,0,0,0],
          [6,0,0,1,9,5,0,0,0],
          [0,9,8,0,0,0,0,6,0],
          [8,0,0,0,6,0,0,0,3],
          [4,0,0,8,0,3,0,0,1],
          [7,0,0,0,2,0,0,0,6],
          [0,6,0,0,0,0,2,8,0],
          [0,0,0,4,1,9,0,0,5],
          [0,0,0,0,8,0,0,7,9]]

solve(puzzle)

count of reductions 4
[[5, 3, 4, 6, 7, 8, 9, 1, 2], [6, 7, 2, 1, 9, 5, 3, 4, 8], [1, 9, 8, 3, 4, 2, 5, 6, 7], [8, 5, 9, 7, 6, 1, 4, 2, 3], [4, 2, 6, 8, 5, 3, 7, 9, 1], [7, 1, 3, 9, 2, 4, 8, 5, 6], [9, 6, 1, 5, 3, 7, 2, 8, 4], [2, 8, 7, 4, 1, 9, 6, 3, 5], [3, 4, 5, 2, 8, 6, 1, 7, 9]]


In [41]:
problem = [
    [9, 0, 0, 0, 8, 0, 0, 0, 1],
    [0, 0, 0, 4, 0, 6, 0, 0, 0],
    [0, 0, 5, 0, 7, 0, 3, 0, 0],
    [0, 6, 0, 0, 0, 0, 0, 4, 0],
    [4, 0, 1, 0, 6, 0, 5, 0, 8],
    [0, 9, 0, 0, 0, 0, 0, 2, 0],
    [0, 0, 7, 0, 3, 0, 2, 0, 0],
    [0, 0, 0, 7, 0, 5, 0, 0, 0],
    [1, 0, 0, 0, 4, 0, 0, 0, 7]
]
solve(problem)

count of reductions 2
count of reductions 1
count of reductions 1
count of reductions 2
count of reductions 4
count of reductions 1
count of reductions 1
count of reductions 5
[[9, 2, 6, 5, 8, 3, 4, 7, 1], [7, 1, 3, 4, 2, 6, 9, 8, 5], [8, 4, 5, 9, 7, 1, 3, 6, 2], [3, 6, 2, 8, 5, 7, 1, 4, 9], [4, 7, 1, 2, 6, 9, 5, 3, 8], [5, 9, 8, 3, 1, 4, 7, 2, 6], [6, 5, 7, 1, 3, 8, 2, 9, 4], [2, 8, 4, 7, 9, 5, 6, 1, 3], [1, 3, 9, 6, 4, 2, 8, 5, 7]]
